In [1]:
# libraries 

from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
import time
from selenium.webdriver.chrome.service import Service

In [8]:
s = Service(r"C:\Users\Hebert Palmera\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")

driver = webdriver.Chrome(service=s)

driver.get("https://www.linkedin.com/jobs/search/?alertAction=viewjobs&currentJobId=3822150905&f_SB2=1&f_TPR=a1705895389-&f_WT=1%2C2%2C3&geoId=106981407&keywords=operations%20analyst&location=Ohio%2C%20United%20States&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=R")

jobs_listing = driver.find_elements(By.XPATH, '//*[@id="main-content"]/section[2]/ul/li')

for listing in jobs_listing:
    job_title_element = listing.find_element(By.CLASS_NAME, 'base-search-card__title')
    job_title = job_title_element.text
    print("Job Title:", job_title)

Job Title: Transportation/I-Trade Operations Analyst
Job Title: Operations Manager - Manufacturing
Job Title: Operations Manager - Manufacturing
Job Title: SI&OP Senior Analyst
Job Title: Sales Operations Analyst
Job Title: SQL Reporting Analyst
Job Title: Operations Manager
Job Title: Business Process Analyst
Job Title: Business Analyst
Job Title: Warehouse Operations Manager
Job Title: Operations Manager - Manufacturing
Job Title: Business Analyst Kronos
Job Title: Operations Manager - Manufacturing
Job Title: Operations Manager - Manufacturing
Job Title: Supply Chain Analyst
Job Title: Business Analyst - Entry Level
Job Title: Sr Analyst - Restaurant People Safety/ Accident Prevention
Job Title: Senior Business Analyst
Job Title: Mortgage Operations Manager, Top US Builder
Job Title: Senior Business Analyst
Job Title: Business Analyst
Job Title: Manager II, Distribution Operations
Job Title: Business Process Analyst 2 - HYBRID
Job Title: Machine Tool Operations Manager - Plastics (H